In [1]:
import pandas as pd
import re
from random import randint
import spacy
from time import time

# Folketingstidende datawrangling

Det følgende script er en renskrivning af et "trial and error" script jeg tidligere har udarbejdet. Dette script kan findes på min github:

### Formål
Dette script ønsker at formatere "folketingstidende" på en sådan måde at der kan laves en digitalhistorisk analyse af data. Formålet med scriptet er da at "genkende" når en ny person begynder at tale fra folketingsets talerstol, så folektingstidende kan opdeles i folketingstaler og udtræke metadateen "hvem der taler" samt "hvilket parti tilhører taleren". Derudover skal scriptet søge at genkende når der er "fejl" i data som skal sorteres fra. Her er primært tale om at sortere marginer fra. Da der er tale om en "OCR" læsning af folketingstidende starter hver side med marginen på siden, som hvis denne ikke sorteres fra, vil indgå som en del af et folketingsmedlem eller ministers tale. 

### Genkendelse af talere:

I det følgende vil jeg søge at benytte mig af regex til at genkende når en ny person indtager folketingsets talerstol. Til dette vil jeg udnytte hvordan folketingstidende er "formateret" af udgiver. Når en ny person indtager folketingets talerstol vil dette blive skrevet som "Navn på talende folketingsmedlem (Parti som folketingsmedlem tilhører)". Det er altså dette jeg vil søge at genkende.

Jeg starter med at indlæse en del af folketingstidende som prøve.

In [2]:
df = pd.read_csv('../../Folketingstidende/Hele/19801.csv',sep=';', encoding = "latin1")

Jeg finder ny søjlenavnene:

In [3]:
df.columns.values.tolist()

['Unnamed: 0', 'spalte', 'TextBlock', 'TopMargin', 'indhold', 'år', 'samling']

Det er søjlen "indhold" der primært har interesse.

In [4]:
FT_data = [indhold for indhold in df["indhold"]]

In [5]:
FT_data[0:10]

['(1)',
 '1',
 '7/10 80: Folketingets åbning',
 '2',
 '1. møde',
 "Tirsdag den 7. oktober 1'980 kl. 12.00",
 'Aldersformanden (Ninn-Hansen):',
 'I henhold til grundloven er folketinget i dag trådt sammen til sit første møde i det nye folketingsår.',
 'Som det medlem, der længst har haft sæde i tinget, påhviler det mig at lede valget af tingets præsidium og tingsekretærer.',
 'Til formand har en gruppe af tingets medlemmer omfattende socialdemokratiets, venstres, det konservative folkepartis, socialistisk folkepartis, det radikale venstres, centrumdemokraternes og kristeligt folkepartis medlemmer indstillet den hidtidige formand, K. B. Andersen (S). Der foreligger ikke andre indstillinger. Hvis ingen begærer afstemning, vil jeg betragte indstillingen som vedtaget. (Ophold). Den er vedtaget.']

Her viser jeg de første 10 OCR-læste afsnit i folketingstidende.

Jeg vil nu søge at genkende når der er folk der taler fra folketingsets talerstol.

In [6]:
Prøve_FT = []
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"] #liste af partier
for sentence in FT_data:
    #if len(re.findall("(\w{1,10}\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0:
        if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
            if len(re.findall("Ny stedf.", sentence)) > 0:
                pass
            elif len(re.findall("Udtræder:", sentence)) > 0:
                pass
            elif len(re.findall("Nyt medl", sentence)) > 0:
                pass
            elif len(re.findall(" af", sentence.lower())) > 0:
                pass
            elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                Prøve_FT.append(sentence)

Jeg kan nu undersøges hvad der genkendes:

In [7]:
Prøve_FT.sort(key = len, reverse = True)

In [8]:
len(Prøve_FT)

9346

In [9]:
Prøve_FT[0:10]

['I overenstemmelse med en henstilling, som jeg har modtaget fra kulturudvalget, hvortil er henvist forslag til folketingsbeslutning vedrørende videokassettebånd med voldsfilm [af Poul H. Møller (KrF) m. fl.] (B 75), skal jeg foreslå, at det nævnte forslag i stedet henvises til retsudvalget. Hvis ingen gør indsigelse, vil jeg betragte dette forslag som vedtaget. (Ophold). Det er vedtaget.',
 'Johan Philipsen (V): »Meningen med, at vi nu får et stort indvandsreservoir med fladt vand, er jo, at vi får en enestående lejlighed til at lave et fuglereservat i dette land.«',
 'Forslag til folketingsbeslutning vedrørende videokassettebånd med voldsfilm [af Poul H. Møller (KrF) m. fl.].',
 'Forslag til folketingsbeslutning om forbud mod reklamering for stærke drikke [af Inge Krogh (KrF) m. fl.].',
 'Mørch (S) (se foran) ? vedtoges med 82 stemmer mod 56; 5 stemte hverken for eller imod.',
 'Forslag til folketingsbeslutning om reagensglasbørn [af Inge Krogh (KrF) m. fl.].',
 'Forslag til folketin

Det ovenstående søger at genkende når der er et menigt medlem(hermed ment ikke formanden eller en minister) som ønsker at tale. Problemet er her at opremsningen af medlemmer af folketinget i starten af folketingsåret genkendes som "en taler" samt at metatale lavet af formanden genkendes eks. "Medlem af folketinget Askjær Jørgensen (FP) har meddelt mig, at han ønsker til industriministeren at stille følgende:". I begge tilfælde ser jeg det dog som mindre problematisk. 
I første tilfælde vil de "genkendte" talere blive tildelt en "tale" som er tom, altså at disse ikke siger noget, hvilket vil blive sorteret fra når jeg udtrækker de folketingstaler jeg ønsker at analysere. 
I andet tilfælde af problemet er ligeledes mindre problematisk. Her er der tale om at folketingets formand fremsætter et menigt medlems holdninger, og deraf ser jeg det ikke problematisk at "karakterisere" disse som folketingstaler.

### Genkendelse af ministre:

In [12]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [13]:
ministre_FT.sort(key = len, reverse = True)

In [14]:
ministre_FT[0:10]

['Administrator Bent Collin, Nationalmuseet, repræsentant for kulturministeriet (formand)',
 "Ministeren for kulturelle anliggender (Lise Østergaard): \\'",
 'Ministeren for offentlige arbejder (Jens Risgaard Knudsen):',
 'Ministeren for offentlige arbejder (Jens Risgaard Knudsen):',
 'Ministeren for kulturelle anliggender (Lise 0stergaard) :',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):',
 'Ministeren for kulturelle anliggender (Lise Østergaard):']

Det ovenstående genkender meget godt når der er tale om en minister.

### Genkendelse af Formanden:

In [15]:
count_formanden = 0
for sentence in FT_data:
    if sentence.lower() == "formanden" or sentence.lower() == "formanden:":
        count_formanden += 1
count_formanden

1024

Jeg undersøger nu om der kan være nogle datapunkter der er blevet misset:

In [16]:
formanden_problem = []
for sentence in FT_data:
    if len(sentence) <= 15:
        if len(re.findall("formanden", sentence.lower())) > 0:
            if sentence.lower() != "formanden" and sentence.lower() != "formanden:":
                formanden_problem.append(sentence)
formanden_problem[0:10]

['[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]']

I det ovenstående er der tale om at der står i margenen af folketingstidende, at det er formanden der taler, det er derfor ikke problematisk at disse sorteres fra.

### Marginer sorteres fra:

In [17]:
hele_margin = []
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
        if len(sentence) < 200:
             hele_margin.append(sentence)
print(len(hele_margin))
hele_margin[0:10]

2517


['11 7/10 80: Statsministerens åbningstale 12',
 '15 7/10 80: Statsministerens åbningstale 16',
 '17 7/10 80: Statsministerens åbningstale IR',
 '19 7/10 80: Statsministerens åbningstale 20',
 '21 7/10 80: Statsministerens åbningstale. Skr. del 22',
 '23 7/10 80: Statsministerens åbningstale. Skr. del 24',
 '35 7/10 80: Statsministerens åbningstale. Skr. del 36',
 '39 7/10 80: Statsministerens åbningstale. Skr. del 40',
 '41 7/10 80: Statsministerens åbningstale. Skr. del 11 42',
 '43 7/10 80: Statsministerens åbningstale. Skr. del 44']

Det er klart at det ovenstående er marginerne på hver side. Hvis man dog følger tallet til venstre, som er det første spalte(af to) på hver side af folketingstidende, kan man dog se at det ikke er alle marginer der findes her.

In [18]:
dato_alene = []
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                dato_alene.append(sentence)
print(len(dato_alene))
dato_alene[0:10]

1970


['7/10 80: Folketingets åbning',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbningstale',
 '7/10 80: Statsministerens åbnìngstale. Skr. del 26',
 '7/10 80: Statsministerens åbningstale. Skr. del 28',
 '7/10 80: Statsministerens åbningstale. Skr. del',
 '7/10 80: Statsministerens åbningstale. Skr. del']

Her kan det også ses at der er tale om marginer. Disse skal derfor ligeledes sorteres fra. Problemet er her at spaltenumrene da er sorteret som individuelle datapunkter. Jeg prøver da også at sortere disse fra.

In [19]:
tal_genkendt = []
for sentence in FT_data:
    if re.fullmatch("[1-9]{1,10}", sentence.lower()): #tilføjet til at fange headers på sider
        tal_genkendt.append(sentence)
tal_genkendt[0:10]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '13']

Hvis man kigger tallene igennem er det meget plausibelt at der her er tale om de manglende spaltenumre. Disse skal altså også sorteres fra.

I marginerne optræder altså også en navn på vedkommende der er den "tallende" når der skiftes spalte. Disse vil jeg også forsøge at sortere fra, da jeg ikke ønsker at en taler fra folketingets talerstols eget navn skal indgå i dennes tale. Heldigvis har disse navne også en relativt nem formatering, nemlig \[navn\]. Jeg søger altså efter datapunker der matcher denne formatering.

In [20]:
tallere_fra_forrige_side = []
for sentence in FT_data:
    if re.fullmatch("\[.{1,100}\]", sentence.lower()):
        tallere_fra_forrige_side.append(sentence)
print(len(tallere_fra_forrige_side))

6039


In [21]:
tallere_fra_forrige_side[0:10]

['[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]']

### Tallere fra forrige side som ikke er blevet læst som seperate enheder:

In [39]:
tallere_fra_forrige_side_læst_forkert = []
for sentence in FT_data:
    if not re.fullmatch("\[.{1,100}\]", sentence.lower()):
        if not re.fullmatch("\[.{1,100}\].", sentence.lower()):
            if re.match("^\[.{1,100}\]", sentence.lower()):
                tallere_fra_forrige_side_læst_forkert.append(sentence)
print(len(tallere_fra_forrige_side_læst_forkert))

7


In [41]:
tallere_fra_forrige_side_læst_forkert

['[Vedr. stedfortræder for Arentoft]. (Beslutningsforslag nr. B 42).',
 '[Energiministeren] x I om Ekofiskfeltet i 1965, som hr. Svend Heiselberg her gør det.',
 '[Ministeren for offentlige arbejder] indvandsreservoir, og ministeriet for offentlige arbejder har i overensstemmelse hermed den principielle opfattelse, at etablering af et saltvandsindtag derfor kun bør ske på grundlag af en særlig lov.',
 "[Dyremose] '--- vil have ministerens forklaring på, hvorfor man selv i en sag som denne, der tilsyneladende er lille, giver folketinget en meget mangelfuld og derfor i en vis udstrækning fejlagtig information, og hvorfor man så, når man har begået en fejl i foråret, skjuler dette ved i bemærkningerne til det aktuelle lovforslag blot at anføre, at ministeriet for skatter og afgifter har oplyst, at der er behov for en ændring, uden at fortælle, at det i virkeligheden er, fordi man har begået en fejl i regeringen.",
 '[Undervisningsministeren] køkkenassistentuddannelsen, nemlig mod arbejde 

Her findes de forskellige navne fra marginerne, som skal sorteres fra.

## Sammensætning af det ovenstående

I det følgende vil jeg sætte alt det ovenstående sammen, med det formål at udtrække folketingstaler, samt hvem der taler, og sortere marginer fra.

In [22]:
indhold = FT_data
talere = []
tale = []
parti = []
count_folketing = 0
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence

Jeg laver nu en funktion ud af dette:

In [23]:
def FT_taler(indhold):
    talere = []
    tale = []
    parti = []
    count_folketing = 0
    Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
    for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence
    return tale, talere, parti

Ovenstående funktion danner basis for formatering af data til senere analyse.